# Test Gauss-Newton plane-based registration

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import os

import planeslam.io as io
from planeslam.general import plot_3D_setup, color_legend
from planeslam.scan import pc_to_scan
from planeslam.registration import extract_corresponding_features, get_correspondences, residual, jacobian

%load_ext autoreload
%autoreload 2

Read in airsim LiDAR and pose data

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

In [ ]:
# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

Extract planes 

In [ ]:
num_scans = len(PC_data)
scans = num_scans * [None]
scans_transformed = num_scans * [None]
for i in range(num_scans):
    scans[i] = pc_to_scan(PC_data[i])

Get correspondences

In [ ]:
idx_1 = 7
idx_2 = 8

# Plot 2 scans
fig = plt.figure(figsize=(15,7))
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
ax2 = fig.add_subplot(1, 2, 2, projection='3d')

ax1.set_box_aspect((np.ptp(PC_data[0][:,0]), np.ptp(PC_data[0][:,1]), np.ptp(PC_data[0][:,2])))
ax2.set_box_aspect((np.ptp(PC_data[0][:,0]), np.ptp(PC_data[0][:,1]), np.ptp(PC_data[0][:,2])))
ax1.set_xlabel("X")
ax1.set_ylabel("Y")
ax2.set_xlabel("X")
ax2.set_ylabel("Y")

scans[idx_1].plot(ax1, show_normals=True)
scans[idx_2].plot(ax2, show_normals=True)

# Color legend
num_colors = max([len(s.planes) for s in scans[0:2]])
color_legend(ax1, num_colors)
color_legend(ax2, num_colors)

In [ ]:
# Extract normals and distances
# Use idx_2 as source and idx_1 as target
#correspondences = get_correspondences(scans[idx_2], scans[idx_1])
correspondences = [(0,0), (1,1), (2,3), (3,4)]
n_s, d_s, n_t, d_t = extract_corresponding_features(scans[idx_2], scans[idx_1], correspondences)

In [ ]:
q = np.array([[     4.865],
        [     0.006],
        [    -0.015],
        [    17.059],
        [     7.844],
        [     1.633]])

r, n_q = residual(n_s, d_s, n_t, d_t, q)
J = jacobian(n_s, n_q)

np.set_printoptions(precision=2)
J

Test Gauss-Newton registration

In [ ]:
# Initial transformation
t = np.array([0, 1, 0])[:,None]
u = np.array([1, 0, 0])[:,None]
theta = 0.1
#q = np.vstack((t, theta*u))
q = np.random.randn(6,1)

# Gauss-Newton
n_iters = 10
lmbda = 1e-8
mu = 5e-1

for i in range(n_iters):
    r, n_q = residual(n_s, d_s, n_t, d_t, q)
    print("loss: ", np.linalg.norm(r)**2)
    J = jacobian(n_s, n_q)
    q = q + mu * np.linalg.inv(J.T @ J + lmbda * np.eye(6)) @ J.T @ r

In [ ]:
q

In [ ]:
# Extract R and t from q
#t_est = q[:3]
t_est = np.zeros((3,1))
R_est = expmap(q[3:].flatten())

source_transformed = scans[0]
source_transformed.transform(R_est, t_est.flatten())
target = scans[1]

ax = plot_3D_setup(PC_data[0])

source_transformed.plot(ax, show_normals=True)
target.plot(ax, show_normals=True)


Test with two planes

In [ ]:
from planeslam.geometry.plane import BoundedPlane
from planeslam.registration import so3_expmap, se3_expmap
from planeslam.scan import Scan
import copy

V1 = np.array([[-1, 1, -1],
              [1, 1, -1],
              [1, 1, 1],
              [-1, 1, 1]])
V2 = np.array([[-1, -1, -1],
              [-1, 1, -1],
              [-1, 1, 1],
              [-1, -1, 1]])
P = Scan([BoundedPlane(V1), BoundedPlane(V2)])

# Ground-truth transformation
t = np.array([0, -0.5, 0])[:,None]
u = np.array([0, 1, 0])[:,None]
theta = np.pi/12
q = np.vstack((t, theta*u))

R = so3_expmap(q[3:].flatten())

print("t: ", t)
print("R: ", R)

In [ ]:
# P is source, Q is target
Q = copy.deepcopy(P)
Q.transform(R, t.flatten())

In [ ]:
ax = plot_3D_setup()
P.plot(ax, color='b')
Q.plot(ax, color='r')

In [ ]:
# n_s = P.normal 
# d_s = np.dot(P.normal.flatten(), P.center)
# n_t = Q.normal 
# d_t = np.dot(Q.normal.flatten(), Q.center)
correspondences = [(0,0), (1,1)]
n_s, d_s, n_t, d_t = extract_corresponding_features(P, Q, correspondences)

In [ ]:
# Initial transformation
# t = np.array([0, 1, 0])[:,None]
# u = np.array([1, 0, 0])[:,None]
# theta = 0.1
# q = np.vstack((t, theta*u))
T = np.eye(4)

# Gauss-Newton
n_iters = 20
lmbda = 1e-8
mu = 0.5

for i in range(n_iters):
    r, n_q = residual(n_s, d_s, n_t, d_t, T)
    print("loss: ", np.linalg.norm(r)**2)
    J = jacobian(n_s, n_q)
    dv = - mu * np.linalg.inv(J.T @ J + lmbda*np.eye(6)) @ J.T @ r
    T = se3_expmap(dv.flatten()) @ T

In [ ]:
# Extract R and t from q
t_est = q[:3]
R_est = expmap(q[3:].flatten())
print("t_est: ", t_est)
print("R_est: ", R_est)

# Apply transformation to source
P_T = copy.copy(P)
P_T.transform(R_est, t_est.flatten())

In [ ]:
plot_P = np.vstack((np.eye(3), -np.eye(3)))
ax = plot_3D_setup(P=plot_P)
P_T.plot(ax, color='b')
Q.plot(ax, color='r')

Test with R as state and updating R thru exponential map

In [ ]:
from planeslam.geometry.plane import BoundedPlane
from planeslam.registration import expmap
from planeslam.scan import Scan
import copy

V1 = np.array([[-1, 1, -1],
              [1, 1, -1],
              [1, 1, 1],
              [-1, 1, 1]])
V2 = np.array([[-1, -1, -1],
              [-1, 1, -1],
              [-1, 1, 1],
              [-1, -1, 1]])
P = Scan([BoundedPlane(V1), BoundedPlane(V2)])

# Ground-truth transformation
t = np.array([0, -0.5, 0])[:,None]
u = np.array([0, 1, 0])[:,None]
theta = np.pi/12
q = np.vstack((t, theta*u))

R = expmap(q[3:].flatten())

print("t: ", t)
print("R: ", R)

# P is source, Q is target
Q = copy.deepcopy(P)
Q.transform(R, t.flatten())

correspondences = [(0,0), (1,1)]
n_s, d_s, n_t, d_t = extract_corresponding_features(P, Q, correspondences)

In [ ]:
# Initial transformation
t0 = np.array([0, 0.1, 0])[:,None]
R0 = np.eye(3)

# Gauss-Newton
n_iters = 5
lmbda = 1e-8

for i in range(n_iters):
    r, n_q = residual(n_s, d_s, n_t, d_t, R0, t0)
    J = jacobian(n_s, n_q)
    dq = -np.linalg.inv(J.T @ J + lmbda * np.eye(6)) @ J.T @ r
    print(r)
    t0 = t0 + dq[:3]
    R0 = expmap(dq[3:].flatten()) @ R0

print("t_est: ", t0)
print("R_est: ", R0)